In [1]:
from numpy import *
from pandas import *
from scipy.stats import *


from pyromat import *
config["unit_pressure"] = "bar"
config["unit_temperature"]="C"
prop_water=get('mp.H2O')

In [2]:
def Turbine(pi,Ti,xi,pe,η=90):
    # Inlet properties to Turbine
#------------------------------------
    if Ti > prop_water.Ts(pi):
        hi = prop_water.h(p=pi, T=Ti)
        si = prop_water.s(p=pi, T=Ti)
    else:
        hi=prop_water.h(p=pi,x=xi)
        si=prop_water.s(p=pi,x=xi)
        
    # Exit properties of Turbine
#------------------------------------
    # isentropic expansion
    ss_e = si
    he_s = prop_water.h(p=pe, s=ss_e)
        
    # Actual exit enthalpy
    he = hi - (η / 100) * (hi - he_s)
        
    # Exit temperature and dryness fraction
    Te, xe = prop_water.T(h=he, p=pe, quality=True)

    # Exit entropy
    se = prop_water.s(p=pe, T=Te)

    h_f = prop_water.hs(p=pe)[0]
    
    return he[0],h_f[0],Te[0],xe[0]

def Pump(pi, pe, η=75):
    # Inlet properties to pump
#------------------------------------
    hi = prop_water.hs(p=pi)[0]
    si = prop_water.ss(p=pi)[0]
        
    # Exit propertis to pump
#------------------------------------
    # isentropic compression
    ss_e = si

    he_s = prop_water.h(p=pe, s=ss_e)

    # Actual exit enthalpy
    he = hi + (he_s - hi) / (η / 100)

    # Actual exit temperatre
    Te = prop_water.T(h=he, p=pe)

    # Actual exit entropy
    se = prop_water.s(p=pe, T=Te)
        
    return he[0]


def Extraction_Pressure(MW,m,N):
    # supply list for m and N
    # Constants for bleed steam
#------------------------------------
    m=array(m)
    N=array(N) 
#------------------------------------
    out=empty(len(m))
    
    for i in range(len(m)):
        out[i]=m[i]*MW+N[i]
        
    return out

In [3]:
df=read_excel("Input.xlsx",sheet_name='load_ext_pr')
MW_=df['MW'].to_numpy()

m_=[]
N_=[]
for i in range(len(df.loc[0])-1):
    p=df[f"p{i+1}"].to_numpy()
    
    lr=linregress(MW_,p)
    m_.append(lr.slope)
    N_.append(lr.intercept)

In [4]:
df1=read_excel("Input.xlsx",sheet_name='Input_data')

# Plant MW
#------------------------------------
MW=df1['MW'][0]

# Inlet pressure and temperature to hpt
#------------------------------------
P_hpt_i=df1['P_hpt_i'][0]
T_hpt_i=df1['T_hpt_i'][0]
T_rh=df1['T_rh'][0]

# Condenser pressure
#------------------------------------
p_cond = df1['p_cond'][0]

# TTD and ETD to FWH's, here for continuity we have also added deaerator
# but for deaerator both will be zero
#------------------------------------
TTD=df1["TTD"].to_numpy()
ETD=df1["ETD"].to_numpy()
# TTD=array([6,5,0,6,2,5])
# ETD=array([3,4,0,5,4,6])
# TTD=array([2.8,2.8,0,0,0,0])
# ETD=array([5.6,5.6,0,5.6,5.6,5.6])

# Specific heat of water
#------------------------------------
cp_w=df1['cp_w'][0]
#lambda x: 3E-07*x**3 - 0.0001*x**2 + 0.0093**x + 4.0296


# number of FWH (including deaerator)
#------------------------------------
n = int(df1['n'][0])

# position of deaerator
#------------------------------------
dea_pos = int(df1['dea_pos'][0])

# Array of extraction pressures
#------------------------------------
p_ext=empty(n)

# updated extraction steam pressure array with the function
# or you can also give it manually
#------------------------------------
p_ext=Extraction_Pressure(MW,m_,N_)

# Array of mass fraction for bleed
#------------------------------------
x=empty(n)

# Array of enthalpy for bleeds
#------------------------------------
hs=empty(n)

# Array of saturation (liquid) enthalpy of bleeded steam
#------------------------------------
hl=empty(n)

# Array of enthalpy of drip from FWH
#------------------------------------
hc=empty(n)
# as there is no drip from deaerator so keeping it zero
hc[dea_pos-1]=0.0

# Array of feed water enthalpy
#------------------------------------
hf=empty(n+2)
# setting up last element of hf to the cep exit enthalpy
hf[-1]=Pump(p_cond,p_ext[dea_pos-1])

# Array for turbine enthalpies (inlet and exit)
#------------------------------------
ht=empty(n+3)

# Turbine inlet enthalpy (HPT inlet and RH exit)
#------------------------------------
hi_hpt=prop_water.h(p=P_hpt_i,T=T_hpt_i)[0]
hi_ipt=prop_water.h(p=p_ext[0],T=T_rh)[0]

In [5]:
# updating arrays for hs and hl
#------------------------------------
for i in range(n):
    
    if i ==0 :
        pi=P_hpt_i
        Ti=T_hpt_i
        xi=1
        pe=p_ext[i]
        
    else :
        pi=p_ext[i-1]
        xi=xe
        pe=p_ext[i]
        if i==1:
            Ti=T_rh
        else:
            Ti =Te

    hs[i],hl[i],Te,xe=Turbine(pi,Ti,xi,pe)
    if i==n-1:
       hi_cond,_,_,_= Turbine(p_ext[-1],Te,xe,p_cond)


In [6]:
# updating array for hf
#------------------------------------
for i in range(len(hf)-1):
    if i<dea_pos-1:
        # print(i)
        hf[i]=hl[i]-TTD[i]*cp_w
    elif i>=dea_pos:
        hf[i]=hl[i-1]-TTD[i-1]*cp_w
    else:
        hf[i]=Pump(p_ext[dea_pos-1],P_hpt_i)

In [7]:
# updating array for hc
#------------------------------------
for i in range(n):
    if i<dea_pos-1:
        # print(i)
        hc[i]=hf[i+1]+ETD[i]*cp_w
    elif i>=dea_pos:
        hc[i]=hl[i]-ETD[i]*cp_w ##### original paper sign -ve
    else:
        hc[i]=0


In [8]:
# Evaluation of mass fractions
#------------------------------------
x_hps=0
x_lps=0
for i in range(n):
    if i<dea_pos-1: # HPH's
        
        if i==0:
            x[i]=(hf[i]-hf[i+1])/(hs[i]-hc[i])
        else:
            x[i]=((hf[i]-hf[i+1])/(hs[i]-hc[i]))+x_hps*(hc[i]-hc[i-1])/(hs[i]-hc[i])

        x_hps +=x[i]
        
    elif i==dea_pos-1: # Deaerator 
        x[i]=((hf[i+1]-hf[i+2])/(hs[i]-hf[i+2]))-x_hps*(hc[i-1]-hf[i+2])/(hs[i]-hf[i+2])
        x_hps +=x[i]  
    else: # LPS's
        if i== dea_pos:
            x[i]=(1-x_hps)*(hf[i+1]-hf[i+2])/(hs[i]-hc[i])
        else:
            x[i]=((1-x_hps)*(hf[i+1]-hf[i+2])/(hs[i]-hc[i]))+x_lps*(hc[i]-hc[i-1])/(hs[i]-hc[i])
            
        x_lps +=x[i]

In [9]:
# updating array of ht
#------------------------------------
for i in range(len(ht)-1):
    if i==0:
        ht[i]=hi_hpt
    elif i==1:
        ht[i]=hs[0]
    elif i==2:
        ht[i]=hi_ipt
    else:
        ht[i]=hs[i-2]
ht[-1]=hi_cond

In [10]:
# Evaluating Turbine work
#----------------------------------
w_t=ht[0]-ht[1]
Σx=0
for i in range(2,len(ht)-1):
    Σx +=x[i-2]
    w_t += (1-Σx)*(ht[i]-ht[i+1])

In [11]:
# Evaluating Pump work (cep, bfp and total)
#----------------------------------
hi_cep=prop_water.hs(p=p_cond)[0][0]
w_cep=(1-x_hps)*(hf[-1]-hi_cep)
w_bfp=hf[dea_pos-1]-hf[dea_pos]
w_p=w_cep+w_bfp
w_net=w_t-w_p

In [12]:
# Heat interactions in Boiler and condenser
#--------------------------------------------

# Condenser heat rejection
q_cond=(1-x_hps-x_lps)*hi_cond+x_lps*hc[-1]-(1-x_hps)*hi_cep

# boiler heat addition
q_boiler=hi_hpt-hf[0] + (1-x[0])*(hi_ipt-hs[0])

# net heat input
q_net = q_boiler-q_cond

In [13]:
# Total property calculations
#----------------------------------------------
m_dot_plant = MW*1000/w_net

Q_boiler = m_dot_plant*q_boiler
Q_cond = m_dot_plant*q_cond
Q_net = m_dot_plant*q_net

W_t = m_dot_plant*w_t
W_p = m_dot_plant*w_p
W_net = m_dot_plant*w_net

m_extract = x*m_dot_plant

η=w_net/q_boiler

In [14]:
# saving data in excel format
#-----------------------------------------------

Hs=[]
for i in range(len(hs)):
    Hs.append(f"hs({i})")
    
data_frame1 = DataFrame({'hs': Hs,'value (kJ/kg)': hs})

Hf=[]
for i in range(len(hf)):
    Hf.append(f"hf({i})")
data_frame2 = DataFrame({'hf': Hf,'value (kJ/kg)': hf})
 
Hc=[]
for i in range(len(hc)):
    Hc.append(f"hc({i})")
data_frame3 = DataFrame({'hc': Hc,'value (kJ/kg)': hc})

Ht=[]
for i in range(len(ht)):
    Ht.append(f"ht({i})")
data_frame4 = DataFrame({'ht': Ht,'value (kJ/kg)': ht})

quantity1 = ['q_cond','q_boiler','w_turbine','w_pump','w_net','q_net','η']
out_value1 = [q_cond,q_boiler,w_t,w_p,w_net,q_net,η*100]
units1 = ['kJ/kg','kJ/kg','kJ/kg','kJ/kg','kJ/kg','kJ/kg', '%']
data_frame5 = DataFrame({'quantity': quantity1, 'value': out_value1, "Unit": units1})


quantity2 = ['Power','m_dot','Q_cond','Q_boiler','W_turbine','W_pump','W_net','Q_net','η']
out_value2 = [MW,m_dot_plant, Q_cond,Q_boiler,W_t,W_p,W_net,Q_net,η*100]
units2 = ['MW','kg/s','kJ','kJ','kJ','kJ','kJ','kJ', '%']
data_frame6 = DataFrame({'quantity': quantity2, 'value': out_value2, "Unit": units2})


ext_n=[]
for i in range(len(x)):
    ext_n.append(f"Ext({i})")
data_frame7 = DataFrame({"Extraction point": ext_n,"x":x, "m_dot (kg/s)":m_extract})



# create a excel writer object
with ExcelWriter("Output.xlsx") as writer:
   
    # use 'to_excel' function and specify the 'sheet_name' and index 
    # to store the dataframe in specified sheet
    data_frame1.to_excel(writer, sheet_name="hs", index=False)
    data_frame2.to_excel(writer, sheet_name="hf", index=False)
    data_frame3.to_excel(writer, sheet_name="hc", index=False)
    data_frame4.to_excel(writer, sheet_name="ht", index=False)
    data_frame5.to_excel(writer, sheet_name="Specific Quantities", index=False)
    data_frame6.to_excel(writer, sheet_name="Total Quantities", index=False)
    data_frame7.to_excel(writer, sheet_name="Extraction masses", index=False)

a=0
a  =(ht[0]-ht[1])
print(a)
a +=(1-x[0])*(ht[2]-ht[3])
print(a)
a +=(1-x[0]-x[1])*(ht[3]-ht[4])
print(a)
a +=(1-x[0]-x[1]-x[2])*(ht[4]-ht[5])
print(a)
a +=(1-x[0]-x[1]-x[2]-x[3])*(ht[5]-ht[6])
print(a)
a +=(1-x[0]-x[1]-x[2]-x[3]-x[4])*(ht[6]-ht[7])
print(a)
a +=(1-x[0]-x[1]-x[2]-x[3]-x[4]-x[5])*(ht[7]-ht[8])
print(a)

In [15]:
len(ht)

9